# Threshold candidate detection

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import ruptures as rpt
import os
import glob
from PIL import Image
from depth_panorama import rays_to_panorama


DATA_FOLDER = r"C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\raw\t1-1"
PANO_FOLDER = r"C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\panos"
CANDIDATES_FOLDER = r"C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\candidates"

os.makedirs(PANO_FOLDER, exist_ok=True)
os.makedirs(CANDIDATES_FOLDER, exist_ok=True)

W, H = 64, 32
MAX_DIST = 10.0

# change this for every type!!
type_id = "t8-1"

## 1. Generate depth panoramas

In [ ]:
def generate_all_panoramas(data_folder, pano_folder):
    """Generate depth panoramas for all points in all curves."""
    csv_files = sorted(glob.glob(os.path.join(data_folder, "curve_*.csv")))
    typology_id = os.path.basename(data_folder)
    
    for csv_file in csv_files:
        filename = os.path.basename(csv_file)
        curve_id = os.path.splitext(filename)[0]
        
        data = np.loadtxt(csv_file, delimiter=",").T  # shape: (N, 2048)
        n_points = data.shape[0]
        
        for i in range(n_points):
            distances = data[i]
            pano = rays_to_panorama(distances, W=W, H=H, max_dist=MAX_DIST)
            
            pano_name = f"{typology_id}_{curve_id}_{i+1:02d}.png"
            pano_path = os.path.join(pano_folder, pano_name)
            Image.fromarray(pano).save(pano_path)
        
        print(f"Generated {n_points} panoramas for {curve_id}")

generate_all_panoramas(DATA_FOLDER, PANO_FOLDER)

## 2. Compute L2 and DSSIM Time Series

In [ ]:
def compute_metrics(pano_folder, curve_id):
    """Compute L2 distance and DSSIM for a single curve."""
    pano_files = sorted(glob.glob(os.path.join(pano_folder, f"*{curve_id}_*.png")))
    
    panos = [np.array(Image.open(f)) for f in pano_files]
    n = len(panos)
    
    l2_dists = []
    dssims = []
    
    for i in range(n - 1):
        img1 = panos[i].astype(float)
        img2 = panos[i + 1].astype(float)
        
        # L2 distance
        l2 = np.linalg.norm(img2 - img1)
        l2_dists.append(l2)
        
        # DSSIM (1 - SSIM)
        ssim_val = ssim(img1, img2, data_range=255.0)
        dssims.append(1 - ssim_val)
    
    return np.array(l2_dists), np.array(dssims)

## 3. Apply PELT changepoint detection

In [ ]:
def detect_changepoints(signal):
    """Apply PELT algorithm with BIC penalty."""
    n = len(signal)
    penalty = np.log(n)  # BIC penalty
    
    algo = rpt.Pelt(model="rbf").fit(signal.reshape(-1, 1))
    changepoints = algo.predict(pen=penalty)
    
    # Remove the last element (end of signal)
    if changepoints[-1] == n:
        changepoints = changepoints[:-1]
    
    return changepoints

## 4. Consensus + export 7-frame windows

In [ ]:
def process_curve(csv_file, pano_folder, candidates_folder):
    """Detect thresholds and export 7-frame windows."""
    filename = os.path.basename(csv_file)
    curve_id = os.path.splitext(filename)[0]
    typology_id = os.path.basename(os.path.dirname(csv_file))
    
    # Load panoramas and compute metrics
    l2_dists, dssims = compute_metrics(pano_folder, curve_id)
    
    # Detect changepoints
    l2_changepoints = detect_changepoints(l2_dists)
    dssim_changepoints = detect_changepoints(dssims)
    
    # Consensus: detected by both methods
    consensus = set(l2_changepoints) & set(dssim_changepoints)
    moderate = (set(l2_changepoints) | set(dssim_changepoints)) - consensus
    
    print(f"{curve_id}: {len(consensus)} high-confidence, {len(moderate)} moderate")
    
    # Export 7-frame windows for consensus thresholds
    pano_files = sorted(glob.glob(os.path.join(pano_folder, f"*{curve_id}_*.png")))
    n = len(pano_files)
    
    for idx in consensus:
        start = max(0, idx - 3)
        end = min(n, idx + 4)
        
        for frame_offset, i in enumerate(range(start, end)):
            src = pano_files[i]
            dst_name = f"{typology_id}_{curve_id}_peak_{idx:03d}_f{frame_offset:02d}.png"
            dst_path = os.path.join(candidates_folder, dst_name)
            
            img = Image.open(src)
            img.save(dst_path)
    
    return {
        "curve_id": curve_id,
        "consensus": list(consensus),
        "moderate": list(moderate),
        "l2_changepoints": list(l2_changepoints),
        "dssim_changepoints": list(dssim_changepoints)
    }

In [ ]:
csv_files = sorted(glob.glob(os.path.join(DATA_FOLDER, "curve_*.csv")))
results = []

for csv_file in csv_files:
    result = process_curve(csv_file, PANO_FOLDER, CANDIDATES_FOLDER)
    results.append(result)

# Save metadata
typology_id = os.path.basename(DATA_FOLDER)
metadata_path = os.path.join(CANDIDATES_FOLDER, f"{typology_id}_metadata.csv")
pd.DataFrame(results).to_csv(metadata_path, index=False)
print(f"\nMetadata saved to {metadata_path}")

## Visualization for the first path

In [ ]:
# Example: Visualize first curve
csv_file = csv_files[0]
curve_id = os.path.splitext(os.path.basename(csv_file))[0]
l2_dists, dssims = compute_metrics(PANO_FOLDER, curve_id)

l2_changepoints = detect_changepoints(l2_dists)
dssim_changepoints = detect_changepoints(dssims)

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(l2_dists, label="L2 Distance", color="blue")
axes[0].scatter(l2_changepoints, l2_dists[l2_changepoints], color="red", label="L2 Changepoints", zorder=5)
axes[0].set_ylabel("L2 Distance")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(dssims, label="DSSIM (1-SSIM)", color="green")
axes[1].scatter(dssim_changepoints, dssims[dssim_changepoints], color="red", label="DSSIM Changepoints", zorder=5)
axes[1].set_ylabel("DSSIM")
axes[1].set_xlabel("Frame Index")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()